In [1]:
"""
Some quick plots for the isolated disk setup.
"""
import numpy as np
import h5py
import glob
from os.path import isfile, isdir
from os import mkdir

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from sphMap import sphMap

import CalcLoadPlotData as clp

In [19]:
def units():
    """ Load/derive the unit system. """

    # get a snapshot
    snap = glob.glob(path + 'snap*.hdf5')[0]

    with h5py.File(snap,'r') as f:
        attrs = dict(f['Header'].attrs)

    # unit system
    u = {k:v for k,v in attrs.items() if 'Unit' in k}

    u['BoxSize'] = attrs['BoxSize']

    # derived units
    u['UnitTime_in_s']       = u['UnitLength_in_cm'] / u['UnitVelocity_in_cm_per_s']
    u['UnitDensity_in_cgs']  = u['UnitMass_in_g'] / u['UnitLength_in_cm']**3
    u['UnitPressure_in_cgs'] = u['UnitMass_in_g'] / u['UnitLength_in_cm'] / u['UnitTime_in_s']**2
    u['UnitEnergy_in_cgs']   = u['UnitMass_in_g'] * u['UnitLength_in_cm']**2 / u['UnitTime_in_s']**2
    u['UnitTemp_in_cgs']     = u['UnitEnergy_in_cgs'] / u['UnitMass_in_g']

    # helpful conversions
    sec_in_year = 3.155693e7
    Msun_in_g = 1.98892e33
    kpc_in_cm = 3.085680e21

    u['UnitTime_in_Gyr'] = u['UnitTime_in_s'] / sec_in_year / 1e9
    u['UnitMass_in_Msun'] = u['UnitMass_in_g'] / Msun_in_g
    u['UnitLength_in_kpc'] = u['UnitLength_in_cm'] / kpc_in_cm

    u['UnitDensity_in_Msun_kpc3'] = u['UnitDensity_in_cgs'] * (u['UnitMass_in_Msun']/u['UnitLength_in_kpc']**3)

    return u

def numpart_vs_time():
    """ Plot number of gas/stars/DM, versus time. """
    snaps = sorted(glob.glob(path + 'snap*.hdf5'))

    # load
    times = []
    numgas = []
    numdm = []
    numstars = []

    for snap in snaps:
        with h5py.File(snap,'r') as f:
            attrs = dict(f['Header'].attrs)

        times.append(attrs['Time'])
        numgas.append(attrs['NumPart_Total'][0])
        numdm.append(attrs['NumPart_Total'][1])
        numstars.append(attrs['NumPart_Total'][4])

    # convert times to Gyr
    u = units()

    times = np.array(times) * u['UnitTime_in_Gyr']

    # plot
    fig, ax = plt.subplots(figsize=(11,8))

    ax.set_xlabel('Time [ Gyr ]')
    ax.set_ylabel('Number of Gas Cells')
    ax.set_yscale('log')
    
    print("min numgas: ", np.min(numgas))
    print("max numgas: ", np.max(numgas))

    ax.plot(times, numgas, '-', lw=2.5, label='Gas')
    ax.plot(times, numstars, '-', lw=2.5, label='Stars')
    ax.plot(times, numdm, '-', lw=2.5, label='DM')

    ax.legend(loc='best')
    fig.savefig(savePath + 'numpart_vs_time.png')
    plt.close(fig)
    
def partmass_vs_time():
    """ Plot mass of gas/stars/DM, versus time. """
    snaps = sorted(glob.glob(path + 'snap*.hdf5'))

    # load
    times = []
    numgas = []
    numdm = []
    numstars = []
    gasMass = []
    starMass = []

    for snap in snaps:
        with h5py.File(snap,'r') as f:
            attrs = dict(f['Header'].attrs)
            partType0 = dict(f['PartType0'])
            gasMasses = partType0["Masses"]
            totalGasMass = np.sum(gasMasses)
            try:
                f['PartType4']
            except KeyError:
                continue
            else:
                partType4 = dict(f['PartType4'])
                starMasses = partType4["Masses"]
                totalStarMass = np.sum(starMasses)
                starMass.append(totalStarMass)

        times.append(attrs['Time'])
        numgas.append(attrs['NumPart_Total'][0])
        numdm.append(attrs['NumPart_Total'][1])
        numstars.append(attrs['NumPart_Total'][4])
        gasMass.append(totalGasMass)
    # convert times to Gyr
    u = units()

    times = np.array(times) * u['UnitTime_in_Gyr']

    # plot
    fig, ax = plt.subplots(figsize=(11,8))

    ax.set_xlabel('Time [ Gyr ]')
    ax.set_ylabel('Mass of Gas Cells [10e10 M_sun]')
    ax.set_yscale('log')

    ax.plot(times, gasMass, '-', lw=2.5, label='Gas Mass')
    #ax.plot(times, dmMass, '-', lw=2.5, label='DM Mass')
    ax.plot(times, starMass, '-', lw=2.5, label='Star Mass')

    ax.legend(loc='best')
    fig.savefig(savePath + 'partMass_vs_time.png')
    plt.close(fig)

def visualize_frames(quantity = None, onlyTopDown = False, visualizeCluster = False):
    """ Create individual images frames, to be combined into a movie, visualizing the gas. 
    Encode with:

    ffmpeg -f image2 -start_number 0 -i frame_%03d.png -vcodec libx264 -pix_fmt yuv420p -crf 19 -an -threads 0 out.mp4

    """
    nbins = 1000
    if(quantity == None):
        vmm = [4.0, 8.0] # log msun/kpc^2
    elif(quantity == "zVel"):
        vmm = [-3, 3]
    elif(quantity == "temp"):
        vmm = [3.5, 6.0]
        
    if(visualizeCluster):
        vmm = [0, 0.6]

        
    if(visualizeCluster):
        size_faceon = [0,96]
        size_edgeon = [0,96]
    else:
        size_faceon = [70,130] # kpc, i.e. center of box
        size_edgeon = [85,115] # kpc, i.e. narrow for edge-on
        

    if not isdir('frames'):
        mkdir('frames')

    # get list of snapshots, and units
    snaps = sorted(glob.glob(path + 'snap*.hdf5'))
    u = units()

    # loop over each snapshot
    for i, snap in enumerate(snaps):
        
        if(i != 300):
            continue
        
        # output file
        filename = savePath + 'frame_%03d.png' % i
        print(i)

        # frame already exists?
        if(overrideOldFrames == False):
            if isfile(filename):
                continue
                
        # load
        with h5py.File(snap,'r') as f:
            pos = f['PartType0']['Coordinates'][()]
            mass = f['PartType0']['Masses'][()]
            dens = f['PartType0']['Density'][()]
            volume = mass / dens
            
            if(quantity == "zVel"):
                velocities = f['PartType0']['Velocities'][()]
                zVelocities = velocities[:,2]
            elif(quantity == "temp"):
                temperatures = clp.getTemperaturesInKelvin(path, i)

        # convert masses from code units to msun, positions to kpc
        pos *= u['UnitLength_in_kpc']
        if(visualizeCluster):
            pos *= 1000
        mass *= u['UnitMass_in_Msun']
        if(quantity == "zVel"):
            zVelocities *= u['UnitVelocity_in_cm_per_s'] /1e5

        if 0:
            # histogram face-on, and edge-on
            extent = [[0,u['BoxSize']],[0,u['BoxSize']]]

            h2d_faceon, _, _ = np.histogram2d(pos[:,0], pos[:,1], weights=mass, bins=nbins, range=extent)
            h2d_edgeon, _, _ = np.histogram2d(pos[:,0], pos[:,2], weights=mass, bins=nbins, range=extent)

        # new: use sphMap to generate projection images
        cell_radius = (volume * 3.0 / (4*np.pi))**(1.0/3.0) # assume spherical
        size = 2.5 * cell_radius # kpc, factor of 2.5 means slightly larger than cell diameter

        boxCen = [u['BoxSize']/2,u['BoxSize']/2,u['BoxSize']/2]
        boxSizeImg = [u['BoxSize'],u['BoxSize'],u['BoxSize']]
        
        if(quantity == "zVel"):
            boxSizeImg[2] /= 10

        # quant: put in zVelocity (1 nbr for each pos)
        
        
        
        if(quantity == None):
            quant = None
        elif(quantity == "zVel"):
            quant = zVelocities
        elif(quantity == "temp"):
            quant = np.float32(temperatures)
            
        h2d_faceon = sphMap(pos, size, mass, quant=quant, axes=[0,1], boxSizeImg=boxSizeImg, 
                            boxSizeSim=0, boxCen=boxCen, nPixels=[nbins,nbins], ndims=3)

        h2d_edgeon = sphMap(pos, size, mass, quant=quant, axes=[2,0], boxSizeImg=boxSizeImg, 
                            boxSizeSim=0, boxCen=boxCen, nPixels=[nbins,nbins], ndims=3)
        

        # normalize by pixel area: [msun per pixel] -> [msun/kpc^2]
        px_area = (boxSizeImg[0]/nbins)**2 # kpc^2

        if(quantity == None):
            h2d_faceon /= px_area
            h2d_edgeon /= px_area

        w_pos_face = np.where(h2d_faceon > 0)
        w_neg_face = np.where(h2d_faceon < 0)
        
        if(onlyTopDown == False):
            w_pos_edge = np.where(h2d_edgeon > 0)
            w_neg_edge = np.where(h2d_edgeon < 0)
            
        h2d_faceon[w_pos_face] = np.log10(h2d_faceon[w_pos_face])
        h2d_faceon[w_neg_face] = -np.log10(-h2d_faceon[w_neg_face])
        
        if(onlyTopDown == False):
            h2d_edgeon[w_pos_edge] = np.log10(h2d_edgeon[w_pos_edge])
            h2d_edgeon[w_neg_edge] = -np.log10(-h2d_edgeon[w_neg_edge])

        # plot
        if(onlyTopDown == False):
            width_ratios =[0.5,0.5]
            if(visualizeCluster):
                width_ratios=[0.5,0.5]
            else:
                width_ratios=[0.63,0.37]
            fig, (ax_left, ax_right) = plt.subplots(nrows=1, ncols=2, figsize=(13,8), width_ratios=width_ratios, dpi=300) # 
        else:
            fig, ax_left = plt.subplots(nrows=1, ncols=1, figsize=(13,8), dpi=300) # 
            
        
        if(visualizeCluster):
            ax_left.set_xlabel('x [pc]')
            ax_left.set_ylabel('y [pc]')
        else:
            ax_left.set_xlabel('x [kpc]')
            ax_left.set_ylabel('y [kpc]')
        ax_left.set_xlim(size_faceon)
        ax_left.set_ylim(size_faceon)

        extent = [0,u['BoxSize'],0,u['BoxSize']]
        
        if(quantity == None):
            colormap = 'inferno'
        elif(quantity == "zVel"):
            colormap = 'bwr'
        elif(quantity == "temp"):
            colormap = 'plasma'
        
        im_left = ax_left.imshow(h2d_faceon, cmap=colormap, extent=extent, aspect=1.0, vmin=vmm[0], vmax=vmm[1])
            
        if(onlyTopDown == False):
            if(visualizeCluster):
                ax_right.set_xlabel('z [pc]')
                ax_right.set_ylabel('x [pc]')
            else:
                ax_right.set_xlabel('z [kpc]')
                ax_right.set_ylabel('x [kpc]')
            ax_right.set_xlim(size_edgeon)
            ax_right.set_ylim(size_faceon) # should match above

            im_right = ax_right.imshow(h2d_edgeon, cmap=colormap, extent=extent, aspect=1.0, vmin=vmm[0], vmax=vmm[1])

            # colorbar and finish plot
        if(onlyTopDown == False):
            cax = make_axes_locatable(ax_right).append_axes('right', size='10%', pad=0.1)
            cb = plt.colorbar(im_right, cax=cax)
        else:
            cax = make_axes_locatable(ax_left).append_axes('right', size='10%', pad=0.1)
            cb = plt.colorbar(im_left, cax=cax)
            
        if(quantity == None):
            cb.ax.set_ylabel('Gas Surface Mass Density [ $\\rm{log10(M_\odot kpc^{-2})}$ ]')
        elif(quantity == "zVel"):
            cb.ax.set_ylabel('Gas Z-Velocity [ $\\rm{log10(km / s)}$ ]')
        elif(quantity == "temp"):
            cb.ax.set_ylabel('Gas temperature [ $\\rm{log10(K)}$ ]')

        fig.savefig(filename)
        plt.close(fig)

In [20]:
overrideOldFrames = True
path = "/vera/ptmp/gc/xboecker/run/4_galaxy/xeno_diskIC/run_diskIC_wdm/runs_settled_no_refinement_ICs/sE_10x_new_boost_10/"
savePath = "/vera/u/xboecker/jupyterNotebooksOutputs/plots/Galaxy/movies/noRefinement_noOutflow_ICs_runs/sE_10x_new_boost_10/highRes/"

#path = "/vera/ptmp/gc/xboecker/run/4_galaxy/xeno_diskIC/run_diskIC_wdm/ICs_10x_settled_no_refinement_no_outflow/"
#savePath = "/vera/u/xboecker/jupyterNotebooksOutputs/plots/Galaxy/SFR_Cooling_XenoSN/movies/noRefinement_noOutflow_settled_ICs/ICs_10x/"
#visualize_frames(quantity = None, onlyTopDown=False)
# quantity: None / "zVel" / "temp"

#path = "/vera/ptmp/gc/xboecker/run/5_SN_cluster_final/3_rad_20pc/output/"
#savePath = "/vera/u/xboecker/jupyterNotebooksOutputs/plots/Cluster/movies/3_rad_20pc/new/"
visualize_frames(onlyTopDown=False, visualizeCluster=False)

#numpart_vs_time()
#partmass_vs_time()

# ffmpeg -f image2 -start_number 0 -i frame_%03d.png -vcodec libx264 -pix_fmt yuv420p -crf 19 -an -threads 0 out.mp4

300
